In [37]:
from itertools import chain
import nltk
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import LabelBinarizer
import sklearn
import pycrfsuite
import pickle
import pdb

In [2]:
data = pickle.load(open('./data/restructured_dataset.pkl', 'rb'))

In [3]:
def process_label(label):
    if label[0] == '/':
        return label.split('/')[1]
    else:
        return label

In [12]:
# another prototype
features = []
labels = []
ids = []

test_nodes = []
minimum_number_of_test_nodes_wanted = 100
treat_as_test_node = False
for citing_id in data['edges'].keys():
    cited_ids = data['edges'][citing_id]
    if len(test_nodes) < number_of_test_nodes_wanted:
        treat_as_test_node = True
    if citing_id in data['features_for_id'].keys() and citing_id in data['label_for_id'].keys():
        for i in range(len(cited_ids)):
            if cited_ids[i] in data['features_for_id'].keys() and cited_ids[i] in data['label_for_id'].keys():
                ids.append([citing_id, cited_ids[i]])
                the_elements = [
                    data['features_for_id'][citing_id], data['features_for_id'][cited_ids[i]]
                ]
                the_labels = [
                    process_label(data['label_for_id'][citing_id]), process_label(data['label_for_id'][cited_ids[i]])
                ]
                features.append(the_elements)
                labels.append(the_labels)
                if treat_as_test_node:
                    test_nodes += [citing_id, cited_ids[i]]
                    test_nodes = list(set(test_nodes))
    treat_as_test_node = False

In [4]:
features = []
labels = []

for citing_id in data['edges'].keys():
    cited_ids = data['edges'][citing_id]
    if citing_id in data['features_for_id'].keys() and citing_id in data['label_for_id'].keys():
        for i in range(len(cited_ids)):
            if cited_ids[i] in data['features_for_id'].keys() and cited_ids[i] in data['label_for_id'].keys():
                the_elements = [
                    data['features_for_id'][citing_id], data['features_for_id'][cited_ids[i]]
                ]
                the_labels = [
                    process_label(data['label_for_id'][citing_id]), process_label(data['label_for_id'][cited_ids[i]])
                ]
                features.append(the_elements)
                labels.append(the_labels)
                

In [5]:
len(data['edges'].keys())

35788

In [15]:
def divide_to_test_and_train(features, labels, test_nodes=None, ids=None):
    if test_nodes is None:
        last_train_index = 80000
        features_train = features[:last_train_index]
        labels_train = labels[:last_train_index]
        features_test = features[last_train_index:]
        labels_test = labels[last_train_index:]
    else:
        assert(ids is not None)
        features_train = []
        labels_train = []
        features_test = []
        labels_test = []
        
        for i in range(len(features)):
            if (ids[i][0] in test_nodes) or (ids[i][1] in test_nodes):
                features_test.append(features[i])
                labels_test.append(labels[i])
            else:
                features_train.append(features[i])
                labels_train.append(labels[i])
    return features_train, labels_train, features_test, labels_test

In [16]:
features_train, labels_train, features_test, labels_test = divide_to_test_and_train(features, labels, test_nodes, ids)

In [21]:
%%time
trainer = pycrfsuite.Trainer(verbose=False)

for xseq, yseq in zip(features_train, labels_train):
    try:
        trainer.append(xseq, yseq)
    except:
        pdb.set_trace()

CPU times: user 1.87 s, sys: 10.6 ms, total: 1.88 s
Wall time: 1.89 s


In [22]:
trainer.set_params({
    'c1': 1.0,   # coefficient for L1 penalty
    'c2': 1e-3,  # coefficient for L2 penalty
    'max_iterations': 50,  # stop earlier

    # include transitions that are possible, but not observed
    'feature.possible_transitions': True
})

In [23]:
trainer.params()

['feature.minfreq',
 'feature.possible_states',
 'feature.possible_transitions',
 'c1',
 'c2',
 'max_iterations',
 'num_memories',
 'epsilon',
 'period',
 'delta',
 'linesearch',
 'max_linesearch']

In [24]:
%%time
trainer.train('cora.crfsuite')

CPU times: user 12.8 s, sys: 7.03 ms, total: 12.8 s
Wall time: 12.8 s


In [25]:
!ls -lh ./cora.crfsuite

-rw-rw-r-- 1 shayan shayan 2.8M Mar 13 15:08 ./cora.crfsuite


In [26]:
trainer.logparser.last_iteration

{'num': 50,
 'scores': {},
 'loss': 44643.792855,
 'feature_norm': 232.584186,
 'error_norm': 513.115247,
 'active_features': 44706,
 'linesearch_trials': 1,
 'linesearch_step': 1.0,
 'time': 0.228}

In [27]:
tagger = pycrfsuite.Tagger()
tagger.open('cora.crfsuite')

In [28]:
from collections import Counter
info = tagger.info()

def print_transitions(trans_features):
    for (label_from, label_to), weight in trans_features:
        print("%-6s -> %-7s %0.6f" % (label_from, label_to, weight))

print("Top likely transitions:")
print_transitions(Counter(info.transitions).most_common(15))

print("\nTop unlikely transitions:")
print_transitions(Counter(info.transitions).most_common()[-15:])

Top likely transitions:
Artificial_Intelligence -> Artificial_Intelligence 3.357085
Programming -> Programming 2.390498
Databases -> Databases 2.346347
NOLABEL -> Artificial_Intelligence 1.915156
Artificial_Intelligence -> NOLABEL 1.846609
Encryption_and_Compression -> Encryption_and_Compression 1.557933
Programming -> NOLABEL 1.444848
NOLABEL -> NOLABEL 1.432596
NOLABEL -> Programming 1.392736
Data_Structures__Algorithms_and_Theory -> Data_Structures__Algorithms_and_Theory 1.302508
Human_Computer_Interaction -> Human_Computer_Interaction 1.287382
Databases -> NOLABEL 1.215583
Programming -> Databases 1.195973
Networking -> Networking 1.051872
NOLABEL -> Databases 1.026220

Top unlikely transitions:
Hardware_and_Architecture -> Networking -0.946496
Networking -> Hardware_and_Architecture -0.954612
Databases -> Hardware_and_Architecture -1.013215
Encryption_and_Compression -> Information_Retrieval -1.028302
Human_Computer_Interaction -> Information_Retrieval -1.034230
Human_Computer_Int

In [34]:
def full_classification_report(y_true, y_pred):
    lb = LabelBinarizer()
    y_true_combined = lb.fit_transform(list(chain.from_iterable(y_true)))
    y_pred_combined = lb.transform(list(chain.from_iterable(y_pred)))
        
    tagset = set(lb.classes_) - {'O'}
    tagset = sorted(tagset, key=lambda tag: tag.split('-', 1)[::-1])
    class_indices = {cls: idx for idx, cls in enumerate(lb.classes_)}
    
    return classification_report(
        y_true_combined,
        y_pred_combined,
        labels = [class_indices[cls] for cls in tagset],
        target_names = tagset,
    )

In [35]:
%%time
y_pred = [tagger.tag(xseq) for xseq in features_test]

CPU times: user 57.7 ms, sys: 87 µs, total: 57.8 ms
Wall time: 56.6 ms


In [38]:
print(full_classification_report(labels_test, y_pred))

                                        precision    recall  f1-score   support

               Artificial_Intelligence       0.88      0.84      0.86      1747
Data_Structures__Algorithms_and_Theory       0.70      0.56      0.63       300
                             Databases       0.74      0.85      0.79       293
            Encryption_and_Compression       0.94      0.58      0.72        50
             Hardware_and_Architecture       0.97      0.47      0.64       159
            Human_Computer_Interaction       0.46      0.54      0.50       151
                 Information_Retrieval       1.00      0.56      0.71        18
                               NOLABEL       0.74      0.76      0.75      2126
                            Networking       0.58      0.53      0.55       257
                     Operating_Systems       0.71      0.72      0.71      1117
                           Programming       0.56      0.79      0.65       326

                             micro avg

In [ ]:
!pip3 install --upgrade pystruct